In [13]:
import boto3
from botocore.exceptions import NoCredentialsError, ClientError
import os
import yaml 

with open('../secrets/aws.yaml', 'r') as file:
    credentials = yaml.safe_load(file)

# Set up AWS credentials
aws_access_key = credentials['aws_access_key_id']
aws_secret_key = credentials['aws_secret_access_key']
region_name = credentials['aws_region']

def create_bucket(bucket_name, region=None):
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)
        print(f'Bucket "{bucket_name}" created successfully.')
    except ClientError as e:
        print(e)
        return False
    return True


def upload_file_to_bucket(bucket_name, file_name, folder_name, object_name=None):
    if object_name is None:
        object_name = os.path.join(folder_name, os.path.basename(file_name))

    s3_client = boto3.client(
        's3',
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key,
        region_name=region_name
    )
    try:
        s3_client.upload_file(file_name, bucket_name, object_name)
        print(f'File "{file_name}" uploaded to bucket "{bucket_name}" in folder "{folder_name}" as "{object_name}".')
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False
    except ClientError as e:
        print(e)
        return False
    return True


In [11]:
# Replace 'your-bucket-name' with your desired bucket name
bucket_name = 'kgto-duckdb-project01'
create_bucket(bucket_name, region_name)

Bucket "kgto-duckdb-project01" created successfully.


True

In [14]:
upload_file_to_bucket(bucket_name, '../seeds/raw_data.csv', 'data', object_name=None)

File "../seeds/raw_data.csv" uploaded to bucket "kgto-duckdb-project01" in folder "data" as "data/../seeds/raw_data.csv".


True